# 附件A：PyTorch的介绍（第二部分）

## A.9 使用GPU优化训练性能

### A.9.1 在GPU上进行 PyTorch 计算

In [13]:
import torch
# 显示PyTorch的版本
print(torch.__version__)

2.5.1+cu124


In [14]:
# 显示PyTorch是否支持GPU
print(torch.cuda.is_available())

True


In [15]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])

print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [16]:
# 将两个张量移动到CUDA设备上
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")

print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


In [17]:
tensor_1 = tensor_1.to("cpu")
print(tensor_1 + tensor_2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

## A.9.2 单GPU训练

In [18]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [19]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [23]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [24]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 第一个隐藏层
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 第二个隐藏层
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # 输出层
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [28]:
import torch.nn.functional as F


# 设置随机数种子，以确保可复现性
torch.manual_seed(123)

# 创建神经网络模型
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# 根据设备可用情况选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device: {device}")


# 如果使用 CUDA，打印设备的详细信息
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
# 将模型移动到所选设备上
model = model.to(device)

# 定义优化器，使用随机梯度下降 (SGD)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

# 定义训练循环的 epoch 数量
num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        features, labels = features.to(device), labels.to(device) 
        logits = model(features)
        loss = F.cross_entropy(logits, labels) # 损失函数

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### 训练日志
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()
    # 可选的模型参数

Current device: cuda
Using GPU: NVIDIA GeForce MX250
Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [29]:
# 使用accuracy（准确率）作为指标
def compute_accuracy(model, dataloader, device):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        # 将数据移动到指定的设备上
        features, labels = features.to(device), labels.to(device) # New

        with torch.no_grad():
            logits = model(features)
        # 获取预测结果并计算准确数量
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
    # 计算并返回准确率
    return (correct / total_examples).item()

In [30]:
import torch

# 检查是否有可用的 GPU
if torch.cuda.is_available():
    # 获取当前 GPU 的显存使用情况
    total_memory = torch.cuda.get_device_properties(0).total_memory  # 总显存
    allocated_memory = torch.cuda.memory_allocated(0)  # 已分配显存
    reserved_memory = torch.cuda.memory_reserved(0)  # 已保留显存

    # 打印显存信息（单位为字节）
    print(f"Total Memory: {total_memory / 1024**3:.2f} GB")
    print(f"Allocated Memory: {allocated_memory / 1024**3:.2f} GB")
    print(f"Reserved Memory: {reserved_memory / 1024**3:.2f} GB")
else:
    print("CUDA is not available")

Total Memory: 2.00 GB
Allocated Memory: 0.02 GB
Reserved Memory: 0.02 GB


In [31]:
compute_accuracy(model, train_loader, device=device)

1.0

In [32]:
compute_accuracy(model, test_loader, device=device)

1.0